In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import torch.optim as optim
import time
import os
image_size = 50

In [5]:
#Network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout2d(p=0.1)
        
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.drop2 = nn.Dropout2d(p=0.15)
        
        self.conv3 = nn.Conv2d(64, 128, 5)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.drop3 = nn.Dropout2d(p=0.15)
        
        x = torch.randn(image_size,image_size).view(-1,1,image_size,image_size)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 2)
        
    def convs(self,x):
        x = self.drop1(self.pool1(F.relu(self.conv1(x))))
        x = self.drop2(self.pool2(F.relu(self.conv2(x))))
        x = self.drop3(self.pool3(F.relu(self.conv3(x))))
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
        
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = self.fc1(x)
        return F.softmax(x, dim=1)

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout2d(p=0.1, inplace=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop2): Dropout2d(p=0.15, inplace=False)
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop3): Dropout2d(p=0.15, inplace=False)
  (fc1): Linear(in_features=10368, out_features=2, bias=True)
)


In [9]:
#data
training_data = np.load("training_data_C.npy", allow_pickle=True)

x = torch.Tensor([i[0] for i in training_data]).view(-1, image_size, image_size)
x = x/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(x)*VAL_PCT)

train_x = x[:-val_size]
train_y = y[:-val_size]

test_x = x[-val_size:]
test_y = y[-val_size:]

np.save("train_x_C.npy", train_x)
np.save("train_y_C.npy", train_y)

np.save("test_x_C.npy", test_x)
np.save("test_y_C.npy", test_y)

RuntimeError: shape '[-1, 3, 100, 100]' is invalid for input of size 187095000

In [ ]:
#Not GPU train and test
BATCH_SIZE = 100 #modify first
EPOCHS = 1

optimizer = optim.Adam(net.parameters(), lr=0.0005, weight_decay=0.01)
loss_f = nn.MSELoss()

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_x), BATCH_SIZE)):
        #print(i, i+BATCH_SIZE)
        batch_x = train_x[i:i+BATCH_SIZE].view(-1,1,image_size,image_size)
        batch_y = train_y[i:i+BATCH_SIZE]
        
        net.zero_grad() # /optimizer.zero_grad()
        
        outputs = net(batch_x)
        loss = loss_f(outputs, batch_y)
        loss.backward()
        optimizer.step()  # Does the update
        
    print(f"Epoch: {epoch} Loss: {loss}")
    
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_x))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_x[i].view(-1,1,image_size,image_size))[0]
        pre = torch.argmax(net_out)
        if pre == real_class:
            correct += 1
        total += 1
print("Acc: ", round(correct/total, 3))

In [3]:
#Connect to gpu, Problem: used up dedicated GPU memory usage, soln: restart kernel
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    a = torch.cuda.device_count()
    net = Net().to(device) #/ net.to(device)
    print(f"Connecting {a} GPU to the Neural Network")
else:
    device = torch.device("cpu")

Connecting 1 GPU to the Neural Network


In [ ]:
#GPU train and test
def train(net):
    optimizer = optim.Adam(net.parameters(), lr=0.00005, weight_decay=1e-9)
    loss_f = nn.MSELoss()
    BATCH_SIZE = 100
    EPOCHS = 10
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_x), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_x = train_x[i:i+BATCH_SIZE].view(-1, 1, image_size, image_size)
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_x, batch_y = batch_x.to(device), batch_y.to(device) # !!!!!!!!
            net.zero_grad()

            optimizer.zero_grad()   # zero the gradient buffers
            outputs = net(batch_x)
            loss = loss_f(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update

        print(f"Epoch: {epoch} Loss: {loss}")

train(net)

BATCH_SIZE = 100
EPOCHS = 10

correct = 0
total = 0
for i in tqdm(range(0, len(test_x), BATCH_SIZE)):

    batch_test_x = test_x[i:i+BATCH_SIZE].view(-1, 1, image_size, image_size).to(device)
    batch_test_y = test_y[i:i+BATCH_SIZE].to(device)
    batch_out = net(batch_test_x)

    out_maxes = [torch.argmax(i) for i in batch_out]
    target_maxes = [torch.argmax(i) for i in batch_test_y]
    for i,j in zip(out_maxes, target_maxes):
        if i == j:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

In [6]:
#GPU recorded train and test
train_x = torch.Tensor(np.load("train_x.npy", allow_pickle=True))
train_y = torch.Tensor(np.load("train_y.npy", allow_pickle=True))

test_x = torch.Tensor(np.load("test_x.npy", allow_pickle=True))
test_y = torch.Tensor(np.load("test_y.npy", allow_pickle=True))

m_name = f"model-{int(time.time())}"

net = Net().to(device) # !!!!!!!!
optimizer = optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-8)
loss_f = nn.MSELoss()

print(m_name)

def fwd_pass(x, y, train=False):
    if train:
        net.zero_grad()
        
    outputs = net(x)
    matches = [torch.argmax(i) == torch.argmax(j) for i, j in zip(outputs,y)]
    acc = matches.count(True)/len(matches)
    loss = loss_f(outputs, y)
    
    if train:
        loss.backward()
        optimizer.step()
        
    return acc, loss

def test(size=32):
    random_start = np.random.randint(len(test_x)-size)
    x, y = test_x[random_start:random_start+size], test_y[random_start:random_start+size]
    
    with torch.no_grad():
        val_acc, val_loss = fwd_pass(x.view(-1,1,image_size,image_size).to(device), y.to(device))
        
    return val_acc, val_loss

def train():
    TEST_SIZE = 100
    BATCH_SIZE = 100 #network big, batch size small #even number
    EPOCHS = 10
    
    if os.path.exists('./model_result.log') != 1:
        with open("model_result.log", "a") as f:
            f.write("name,times,acc,loss,val_acc,val_loss\n")
            
    with open("model_result.log", "a") as f:
        for epoch in range(EPOCHS):
            for i in tqdm(range(0, len(train_x), BATCH_SIZE)):
                batch_x = train_x[i:i+BATCH_SIZE].view(-1, 1, image_size, image_size)
                batch_y = train_y[i:i+BATCH_SIZE]

                batch_x, batch_y = batch_x.to(device), batch_y.to(device) # !!!!!!!!
                
                acc, loss = fwd_pass(batch_x, batch_y, train=True)
                if i % (BATCH_SIZE/2) == 0:
                    val_acc, val_loss = test(size=TEST_SIZE)
                    f.write(f"{m_name},{round(time.time(),3)}, {round(float(acc),2)}, {round(float(loss),4)}, {round(float(val_acc),2)}, {round(float(val_loss),4)}\n")
                    
            print(f"Epoch: {epoch}, val_loss: {val_loss}")

    with open("model_name.log", "a") as f:
        f.write(f"{m_name}\n\n{net} + {optimizer}\n\n")

train()

  0%|          | 0/225 [00:00<?, ?it/s]

model-1595323500


  0%|          | 0/225 [00:00<?, ?it/s]

Epoch: 0, val_loss: 0.22505420446395874


  0%|          | 0/225 [00:00<?, ?it/s]

...

Epoch: 7, val_loss: 0.14321857690811157


  0%|          | 0/225 [00:00<?, ?it/s]

Epoch: 8, val_loss: 0.14755116403102875


100%|██████████| 225/225 [01:02<00:00,  3.63it/s]

Epoch: 9, val_loss: 0.09875667095184326
